In [1]:
#Basic Libraries
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
#Load Dataset
df=pd.read_csv('/home/smriti/Desktop/NLP/MITACS/Anxiety/Data/CSV/Academic/Acad2012.csv')
#getting rid of NaN
df=df.replace(np.nan, '', regex=True)
#Combining title and text
df["Text"] = df["Title"] + df["Abstract"]
#Now that we don't need Title or Text, we drop those columns before saving the file
df=df.drop(['Title', 'Abstract'], axis = 1)

/home/smriti/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [4]:
df=df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df.head()

,StoreId,documentType,year,Text
0,1322257202,"Journal , Peer Reviewed Journal , Journal Art...",2012,Sleep quality and its psychological correlates...
1,1640006791,"Journal , Peer Reviewed Journal , Journal Art...",2012,The effectiveness of cognitive behavioral ther...
2,1322258832,"Journal , Peer Reviewed Journal , Journal Art...",2012,Psychometric properties of the Cardiac Depress...
3,1640005981,"Journal , Peer Reviewed Journal , Journal Art...",2012,Single-prolonged stress induces apoptosis in d...
4,1640006982,"Journal , Peer Reviewed Journal , Journal Art...",2012,Problem-solving versus cognitive restructuring...


In [6]:
# Convert to list
data=df.Text.values.tolist()
# Remove new line characters
data=[re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data=[re.sub("\'", "", sent) for sent in data]

In [7]:
#Function to clean up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 

In [8]:
#Clean up text
data_words=list(sent_to_words(data))
print(data_words[:1])

[['sleep', 'quality', 'and', 'its', 'psychological', 'correlates', 'among', 'university', 'students', 'in', 'ethiopia', 'cross', 'sectional', 'studybackground', 'sleep', 'is', 'an', 'important', 'physiological', 'process', 'for', 'humans', 'university', 'students', 'in', 'most', 'resource', 'limited', 'countries', 'often', 'report', 'poor', 'sleep', 'quality', 'due', 'to', 'changing', 'social', 'opportunities', 'and', 'increasing', 'academic', 'demands', 'however', 'sleep', 'quality', 'among', 'university', 'students', 'has', 'not', 'been', 'studied', 'in', 'ethiopia', 'thus', 'this', 'study', 'assessed', 'sleep', 'quality', 'and', 'its', 'demographic', 'and', 'psychological', 'correlates', 'among', 'university', 'students', 'methods', 'cross', 'sectional', 'survey', 'was', 'conducted', 'in', 'two', 'universities', 'in', 'ethiopia', 'multistage', 'sampling', 'procedures', 'were', 'used', 'to', 'enroll', 'students', 'into', 'the', 'study', 'self', 'administered', 'structured', 'question

In [9]:
# Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram=gensim.models.Phrases(bigram[data_words], threshold=100)

In [10]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod=gensim.models.phrases.Phraser(trigram)

In [11]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['sleep', 'quality', 'and', 'its', 'psychological', 'correlates', 'among', 'university', 'students', 'in', 'ethiopia', 'cross_sectional', 'studybackground', 'sleep', 'is', 'an', 'important', 'physiological', 'process', 'for', 'humans', 'university', 'students', 'in', 'most', 'resource', 'limited', 'countries', 'often', 'report', 'poor', 'sleep', 'quality', 'due', 'to', 'changing', 'social', 'opportunities', 'and', 'increasing', 'academic', 'demands', 'however', 'sleep', 'quality', 'among', 'university', 'students', 'has', 'not', 'been', 'studied', 'in', 'ethiopia', 'thus', 'this', 'study', 'assessed', 'sleep', 'quality', 'and', 'its', 'demographic', 'and', 'psychological', 'correlates', 'among', 'university', 'students', 'methods', 'cross_sectional', 'survey', 'was', 'conducted', 'in', 'two', 'universities', 'in', 'ethiopia', 'multistage', 'sampling', 'procedures', 'were', 'used', 'to', 'enroll', 'students', 'into', 'the', 'study', 'self', 'administered', 'structured', 'questionnaire',

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

# Create Dictionary needed for topic modelling
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=38,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

[['sleep', 'quality', 'psychological', 'correlate', 'university', 'student', 'studybackground', 'sleep', 'important', 'physiological', 'process', 'human', 'university', 'student', 'resource', 'limited', 'country', 'often', 'report', 'poor', 'sleep', 'quality', 'due', 'change', 'social', 'opportunity', 'increase', 'academic', 'demand', 'however', 'sleep', 'quality', 'university', 'student', 'study', 'thus', 'study', 'assess', 'quality', 'demographic', 'psychological', 'correlate', 'university', 'student', 'method', 'survey', 'conduct', 'university', 'multistage', 'sampling', 'procedure', 'use', 'enroll', 'student', 'study', 'self', 'administer', 'structure', 'questionnaire', 'include', 'index', 'psqi', 'depression', 'anxiety', 'scale', 'perceive', 'stress', 'select', 'step', 'instrument', 'use', 'study', 'research', 'include', 'student', 'mean', 'use', 'characterize', 'sleep', 'quality', 'variable', 'analysis', 'variance', 'binary', 'procedure', 'also', 'use', 'result', 'prevalence', 'p

In [15]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(8,
  '0.000*"patient" + 0.000*"anxiety" + 0.000*"cognitive" + 0.000*"treatment" + '
  '0.000*"outcome" + 0.000*"depression" + 0.000*"nightmare" + 0.000*"use" + '
  '0.000*"disorder" + 0.000*"therapy"'),
 (34,
  '0.000*"depression" + 0.000*"treatment" + 0.000*"internationally" + '
  '0.000*"problem" + 0.000*"trial" + 0.000*"health" + 0.000*"model" + '
  '0.000*"study" + 0.000*"life" + 0.000*"anxiety"'),
 (22,
  '0.064*"language" + 0.051*"dsm_iv" + 0.026*"asylum" + 0.022*"traumatic" + '
  '0.012*"proficiency" + 0.010*"arrive" + 0.010*"mini" + 0.010*"arrival" + '
  '0.005*"screening" + 0.005*"postmigration"'),
 (23,
  '0.051*"preventive" + 0.047*"cidi" + 0.036*"screening" + '
  '0.021*"participation" + 0.020*"threshold" + 0.016*"refrain" + 0.013*"scope" '
  '+ 0.000*"prevent" + 0.000*"screen" + 0.000*"assess"'),
 (32,
  '0.062*"single" + 0.035*"nucleus" + 0.026*"rat" + 0.026*"apoptosis" + '
  '0.018*"neuron" + 0.017*"peak" + 0.015*"reduce" + 0.013*"tem" + 0.013*"pon" '
  '+ 0.009*"prolo

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -11.371774384544105


In [17]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.39862992922537677


In [18]:
num_topics = 38
topic_words = []
for i in range(num_topics):
    tt = lda_model.get_topic_terms(i,10)
    topic_words.append([id2word[pair[0]] for pair in tt])

In [19]:
for i in range(0,38):
    print(topic_words[i])

['mdd', 'patient', 'generalize', 'disorder', 'relationship', 'linear', 'gain', 'anxiety', 'way', 'rate']
['sleep', 'hybrid', 'monitoring', 'insomnia', 'acceptability', 'diary', 'end', 'ancillary', 'durability', 'isolation']
['mood', 'disorder', 'depressive', 'depressed', 'anxious', 'history', 'relevant', 'anxiety', 'self', 'depression']
['exposure', 'disaster', 'may', 'follow', 'involvement', 'family', 'define', 'brief', 'new', 'effect']
['anxiety', 'worry', 'cognitive', 'review', 'model', 'test', 'consider', 'later', 'work', 'old']
['social', 'participant', 'positive', 'anxiety', 'take', 'cue', 'train', 'reduce', 'control', 'approach']
['effect', 'participant', 'complete', 'act', 'may', 'week', 'clinical', 'affect', 'exercise', 'test']
['ptsd', 'trauma', 'alcohol', 'perceive', 'status', 'consequence', 'rate', 'incremental', 'high', 'patient']
['patient', 'anxiety', 'cognitive', 'treatment', 'outcome', 'depression', 'nightmare', 'use', 'disorder', 'therapy']
['threat', 'bias', 'attenti

In [20]:
df=pd.DataFrame({'Year':[],'Source':[],'Topic_ID':[],'Most_freq_words':[]})
df.head()

,Most_freq_words,Source,Topic_ID,Year


In [21]:
df['Most_freq_words']=topic_words

In [22]:
df = df.assign(Year='2012')
df = df.assign(Source='Academic')
df.shape

(38, 4)

In [23]:
ls=[]
for i in range(0,38):
    ls.append(i)
df['Topic_ID']=ls

In [24]:
df.head()

,Most_freq_words,Source,Topic_ID,Year
0,"[mdd, patient, generalize, disorder, relations...",Academic,0,2012
1,"[sleep, hybrid, monitoring, insomnia, acceptab...",Academic,1,2012
2,"[mood, disorder, depressive, depressed, anxiou...",Academic,2,2012
3,"[exposure, disaster, may, follow, involvement,...",Academic,3,2012
4,"[anxiety, worry, cognitive, review, model, tes...",Academic,4,2012


In [25]:
df.to_csv("topic_words_j2012.csv")